SETUP AND IMPORTS

In [ ]:
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk.tokenize import word_tokenize
import os
import re
import pickle
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [354]:
def getnumber(filename):
    match = re.search(r"file(\d+)\.txt", filename)
    if match:
        integer = int(match.group(1))
        return integer
    else:
        return None

def pre_process_sentence(sentence):
    sentence = sentence.lower()
    tokens = word_tokenize(sentence)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    filtered_tokens = [word for word in filtered_tokens if word.isalnum()]
    filtered_tokens = list(filter(lambda token: token.strip() != '', filtered_tokens))
    return filtered_tokens

def pre_process_file(read_file_path , write_file_path , mode = 'w'):
    with open(write_file_path, 'w') as write_file:
        with open(read_file_path, 'r') as read_file:
            content = read_file.read()
            content_lower = content.lower()         #lowercase the content
            tokens = word_tokenize(content_lower)   # perform tokenization
            stop_words = set(stopwords.words('english'))    # get the stop words
            filtered_tokens_wout_stop_words = [word for word in tokens if word.lower() not in stop_words] # remove the stop words
            filtered_tokens_wout_punc = [word for word in filtered_tokens_wout_stop_words if word.isalnum()] # remove the punctuations
            filtered_tokens = list(filter(lambda token: token.strip() != '', filtered_tokens_wout_punc)) # remove the empty strings

            if(mode == 'p'):
                print("Original Content: ")
                print(content)
                print("-----------------------------------")
                print("Lowercase Content: ")
                print(content_lower)
                print("-----------------------------------")
                print("Tokens: ")
                print(tokens)
                print("-----------------------------------")
                print("Filtered Tokens without stop words: ")
                print(filtered_tokens_wout_stop_words)
                print("-----------------------------------")
                print("Filtered Tokens without punctuations: ")
                print(filtered_tokens_wout_punc)
                print("-----------------------------------")
                print("Filtered Tokens after removal of empty spaces: ")
                print(filtered_tokens)
                print("-----------------------------------")    
                print()
                print()
            if(mode == 'w'):
                write_file.write(' '.join(filtered_tokens)) # write the filtered tokens to the file

def return_files_in_dir(directory_path):
    input_files = []
    with os.scandir(directory_path) as entries:
        for entry in entries:
            if entry.is_file():
                input_files.append(entry.name)    # print(input_files)
    return input_files

In [347]:
read_directory_path = '/Users/utkarshpal/python_projects/IR/text_files'
write_directory_path = '/Users/utkarshpal/python_projects/IR/new_text_files'
# List all files in the directory
input_files = [f for f in os.listdir(read_directory_path) if os.path.isfile(os.path.join(read_directory_path, f))]

In [351]:
for file in input_files[0:5]:
        print(file)
        pre_process_file(read_directory_path+'/'+str(file) , write_directory_path + '/'+str(file) , 'p')

for file in input_files:
    pre_process_file(read_directory_path+'/'+str(file) , write_directory_path + '/'+str(file) , 'w')


file502.txt
Original Content: 
Kit is awesome. I play in my garage just for personal enjoyment not for performances or anything. Once you take the time to break down all the settings, your able to dial in pretty much any kit and sound. With the expansion options and the relatively inexpensive parts expanding is easy and fun.

After a few weeks of daily use for at least an hour a day it still looks and plays beautifully. Overall one of the best purchases I could have made.
-----------------------------------
Lowercase Content: 
kit is awesome. i play in my garage just for personal enjoyment not for performances or anything. once you take the time to break down all the settings, your able to dial in pretty much any kit and sound. with the expansion options and the relatively inexpensive parts expanding is easy and fun.

after a few weeks of daily use for at least an hour a day it still looks and plays beautifully. overall one of the best purchases i could have made.
---------------------

In [330]:
def create_ungigram_inverted_index(read_directory_path):
    input_files = return_files_in_dir(read_directory_path)
    inverted_index = {}
    for file_name in input_files:
        if file_name.endswith('.txt'):  # Check if the file has a .txt extension
            read_file_path = os.path.join(read_directory_path, file_name)
            with open(read_file_path, 'r') as file:
                content = file.read()
                tokens = content.split()
                for token in tokens:
                    if token not in inverted_index:
                        inverted_index[token] = [getnumber(file_name)]
                    else:
                        if getnumber(file_name) not in inverted_index[token]:
                            inverted_index[token].append(getnumber(file_name))
    with open('inverted_index.pickle', 'wb') as handle:
        pickle.dump(inverted_index, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return inverted_index

create_ungigram_inverted_index("/Users/utkarshpal/python_projects/IR/new_text_files")
load_inverted_index = pickle.load(open('inverted_index.pickle', 'rb'))

In [281]:
def search(query_list, operation_list, inverted_index):
    sentence_query=""
    if len(query_list) == 1 :
        sentence_query = query_list[0]
        if query_list[0] in inverted_index:
            return inverted_index[query_list[0]],sentence_query
        else:
            return [],sentence_query
    else:
        result = []
        for i in range(len(query_list)-1):
            if i == 0:
                sentence_query = query_list[i] + " " + operation_list[i] + " " + query_list[i+1]
                if operation_list[i] == 'AND':
                    result = T1_AND_T2(query_list[i], query_list[i + 1], inverted_index)
                elif operation_list[i] == 'OR':
                    result = T1_OR_T2(query_list[i], query_list[i + 1], inverted_index)
                elif operation_list[i] == 'AND NOT':
                    result = T1_AND_NOT_T2(query_list[i], query_list[i + 1], inverted_index)
                elif operation_list[i] == 'OR NOT':
                    result = T1_OR_NOT_T2(query_list[i], query_list[i + 1], inverted_index)
            else:
                sentence_query = sentence_query + " " + operation_list[i] + " " + query_list[i+1]
                if operation_list[i] == 'AND':
                    result = T1_AND_T2(result, query_list[i+1], inverted_index)
                elif operation_list[i] == 'OR':
                    result = T1_OR_T2(result, query_list[i+1], inverted_index)
                elif operation_list[i] == 'AND NOT':
                    result = T1_AND_NOT_T2(result, query_list[i+1], inverted_index)
                elif operation_list[i] == 'OR NOT':
                    result = T1_OR_NOT_T2(result, query_list[i+1], inverted_index)
    
        return result , sentence_query

def T1_AND_T2(term1 , term2 , inverted_index):
    # print(term1)
    # print(term2)
    if type(term1) == str:
        if term1 in inverted_index:
            term1_set = (inverted_index[term1])
        else:
            term1_set = []
    else:
        term1_set = (term1)
    
    if type(term2) == str:
        if term2 in inverted_index:
            term2_set = (inverted_index[term2])
        else:
            term2_set = []
    return list(set(term1_set).intersection(set(term2_set)))
 
def T1_OR_T2(term1 , term2 , inverted_index):
    if type(term1) == str:
        if term1 in inverted_index:
            term1_set = (inverted_index[term1])
        else:
            term1_set = []
    else:
        term1_set = (term1)
    
    if type(term2) == str:
        if term2 in inverted_index:
            term2_set = (inverted_index[term2])
        else:
            term2_set = []

    return list(set(term1_set).union(set(term2_set)))

def T1_AND_NOT_T2(term1 , term2 , inverted_index):
    if type(term1) == str:
        if term1 in inverted_index:
            term1_set = (inverted_index[term1])
        else:
            term1_set = []
    else:
        term1_set = (term1)
    
    if type(term2) == str:
        if term2 in inverted_index:
            term2_set = (inverted_index[term2])
        else:
            term2_set = []
    return list(set(term1_set).difference(set(term2_set)))

def T1_OR_NOT_T2(term1 , term2 , inverted_index):
    if type(term1) == str:
        if term1 in inverted_index:
            term1_set = (inverted_index[term1])
        else:
            term1_set = []
    else:
        term1_set = (term1)
    
    if type(term2) == str:
        if term2 in inverted_index:
            term2_set = (inverted_index[term2])
        else:
            term2_set = []
    universal_set =  (set(sum(inverted_index.values(), [])))
    not_term2_set = (universal_set.difference(set(term2_set)))
    result_set = not_term2_set.union(set(term1_set))
    return list(result_set)


# def T1_OR_NOT_T2(term1 , term2 , inverted_index):
    # if type(term1) == str:
    #     if term1 in inverted_index:
    #         term1_set = (inverted_index[term1])
    #     else:
    #         term1_set = []
    # else:
    #     term1_set = (term1)
    
    # if type(term2) == str:
    #     if term2 in inverted_index:
    #         term2_set = (inverted_index[term2])
    #     else:
    #         term2_set = []
    # union_set = set(term1_set).union(set(term2_set))
    # result_set = union_set.difference(set(term2_set))
    # return list(result_set)

In [360]:
def q4_query():
    n = int(input("Enter the number of queries: "))
    input_query_list = []
    input_operation_list = []

    for i in range(n):
        query = input("Enter the query: ")
        input_query_list.append(query)
        operation = input("Enter the operation: ")
        input_operation_list.append(operation)
    
    for i in range(n):
        query = input_query_list[i]
        operation = input_operation_list[i]
        query_list = pre_process_sentence(query)
        operation_list = [i.strip() for i in operation.split(',')]
        search_result,query_sentence = search(query_list, operation_list, load_inverted_index)
        search_result.sort()
        print(f"Query {i+1}:" , query_sentence)
        print(f"Number of documents retrieved for query {i+1}:" , search_result)
        print(f"Names of documents retrieved for query {i+1}:" , ["file" + str(i) + ".txt" for i in search_result])
    
q4_query()

Query 1: car AND guitar
Number of documents retrieved for query 1: [166, 174, 542]
Names of documents retrieved for query 1: ['file166.txt', 'file174.txt', 'file542.txt']


In [352]:
def create_unigram_positional_index(read_directory_path):
    input_files = return_files_in_dir(read_directory_path)

    positional_index = {}
    for file_name in input_files:
        if file_name.endswith('.txt'): 
            read_file_path = os.path.join(read_directory_path, file_name)
            with open(read_file_path, 'r') as file:
                content = file.read()
                tokens = content.split()
                for i in range(len(tokens)):
                    if tokens[i] not in positional_index:
                        positional_index[tokens[i]] = {getnumber(file_name):[i]}
                    else:
                        if getnumber(file_name) not in positional_index[tokens[i]]:
                            positional_index[tokens[i]][getnumber(file_name)] = [i]
                        else:
                            positional_index[tokens[i]][getnumber(file_name)].append(i)
    new_positional_index = {}
    for key in positional_index:
        new_positional_index[key] = list(sorted(positional_index[key].items()))

    with open('positional_index.pickle', 'wb') as handle:
        pickle.dump(new_positional_index, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return new_positional_index

create_unigram_positional_index("/Users/utkarshpal/python_projects/IR/new_text_files")
load_positional_index = pickle.load(open('positional_index.pickle', 'rb'))

In [332]:
def positional_intersect(term1_list,term2_list,diff):
    result = {}
    i = 0
    j = 0
    while i < len(term1_list) and j < len(term2_list):
        if term1_list[i][0] == term2_list[j][0]:
            term1_positions = term1_list[i][1]
            term2_positions = term2_list[j][1]
            for term1_position in term1_positions:
                for term2_position in term2_positions:
                    if term2_position - term1_position == diff:
                        if term1_list[i][0] not in result:
                            result[term1_list[i][0]] = [term2_position]
                        else:
                            result[term1_list[i][0]].append(term2_position)
            i += 1
            j += 1
        elif term1_list[i][0] < term2_list[j][0]:
            i += 1
        else:
            j += 1

    result = list(result.items())     
    return result


def positional_search(query_list, positional_index):
    result = []
    if len(query_list) == 1:
        return positional_index[query_list[0][0]]
    
    for i in range(len(query_list)-1):
        if query_list[i][0] in positional_index:
            list1 = positional_index[query_list[i][0]]
        else:
            list1 = []
        if query_list[i+1][0] in positional_index:
            list2 = positional_index[query_list[i+1][0]]
        else:
            list2 = []
            
        if i == 0:
            result = positional_intersect(list1,list2,query_list[i+1][1]-query_list[i][1])
        else:
            result = positional_intersect(result, list2,query_list[i+1][1]-query_list[i][1])
    return result

In [363]:
def q5_query():
    n = int(input("Enter the number of queries: "))
    input_query_list = []
    input_operation_list = []

    for i in range(n):
        query = input("Enter the query: ")
        input_query_list.append(query.lower())
    
    for i in range(n):
        query = input_query_list[i] 
        query_list = pre_process_sentence(query)
        query_list_index = [[query_list[i].strip() ,i] for i in range(len(query_list))]

        result = positional_search(query_list_index, load_positional_index)
        print(f"Query {i+1}:" , query)
        print(f"Number of documents retrieved for query {i+1}:" , len(result))
        print(f"Number of documents retrieved for query {i+1}:" , (result))
        print(f"Names of documents retrieved for query {i+1}:" , ["file" + str(i[0]) + ".txt" for i in result])

q5_query()

Query 1: highly recommended
Number of documents retrieved for query 1: 6
Number of documents retrieved for query 1: [(50, [34]), (81, [30]), (171, [10]), (257, [65]), (382, [64]), (626, [64])]
Names of documents retrieved for query 1: ['file50.txt', 'file81.txt', 'file171.txt', 'file257.txt', 'file382.txt', 'file626.txt']
